# Data Cleaning Pipeline
**Team name**: Reneissance Technologies 
**Purpose**: Clean and standardize loan application dataset from multiple sources


## 1. Setup and Imports

In [14]:
import pandas as pd
import numpy as np
import json
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

## 2. Helper Functions

In [15]:
# function for cleaning currency columns
def clean_currency_column(series, decimal_places=None):
    cleaned = (
        series.astype(str)
        .str.replace(r'[\$,]', '', regex=True)
        .str.strip()
    )
    numeric = pd.to_numeric(cleaned, errors='coerce')
    
    if decimal_places is not None:
        return numeric.round(decimal_places)
    else:
        return numeric.astype('Int64')

# function for standardizing categorical values
def standardize_categorical(series, mapping_dict):
    normalized = (
        series.str.lower()
        .str.replace(' ', '_', regex=False)
        .str.replace('-', '_', regex=False)
    )
    return normalized.map(mapping_dict)

# function to show data quality 
def report_data_quality(df, df_name):
    print(f"\n{'='*60}")
    print(f"Data quality report: {df_name}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing': missing[missing > 0],
        'Percentage': missing_pct[missing > 0]
    })
    if len(missing_df) > 0:
        print(missing_df.to_string())
    else:
        print("No missing values")
    print(f"{'='*60}\n")

## 3. Application Metadata Cleaning

In [17]:
metadata = pd.read_csv('application_metadata.csv', dtype=str)

metadata = metadata.rename(columns={'customer_ref': 'customer_id'})

# Standardizing integer and float columns
int_cols = [
    'customer_id', 'application_id', 'application_hour',
    'application_day_of_week', 'account_open_year', 'num_login_sessions',
    'has_mobile_app', 'paperless_billing', 'default'
]
for col in int_cols:
    metadata[col] = pd.to_numeric(metadata[col], errors='coerce').astype('Int64')

metadata['random_noise_1'] = pd.to_numeric(
    metadata['random_noise_1'], errors='coerce'
).round(6)

# Standardize account status codes
metadata['account_status_code'] = metadata['account_status_code'].replace({
    'A01': 'ACT-1',
    'ACTIVE': 'ACT-1'
})

# Check for duplicates
n_dup_app = metadata['application_id'].duplicated().sum()
n_dup_cust = metadata['customer_id'].duplicated().sum()
print(f"Duplicate application_ids: {n_dup_app}")
print(f"Duplicate customer_ids: {n_dup_cust}")

# Quality report
report_data_quality(metadata, "Application Metadata")

metadata.head()

Duplicate application_ids: 0
Duplicate customer_ids: 0

Data quality report: Application Metadata
Shape: 89,999 rows × 14 columns

Missing Values:
No missing values



,customer_id,application_id,application_hour,application_day_of_week,account_open_year,preferred_contact,referral_code,account_status_code,random_noise_1,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing,default
0,10000,620515,5,6,2013,Mail,REF0000,ACT-2,1.137,13,2,1,1,0
1,10001,624978,4,2,2015,Phone,REF0000,ACT-3,-0.165,6,1,1,1,1
2,10002,564658,10,3,2020,Phone,REF0000,ACT-3,0.527,1,2,1,0,0
3,10003,621493,7,5,2010,Email,REF0000,ACT-1,-0.710,4,1,1,1,0
4,10004,637785,1,2,2020,Mail,REF0000,ACT-3,-0.603,6,2,1,0,0


## 4. Demographics Cleaning

In [18]:
demographics = pd.read_csv('demographics.csv')
demographics = demographics.rename(columns={'cust_id': 'customer_id'})

# Cleaning annual income
demographics['annual_income'] = demographics['annual_income'].replace('--', None)
demographics['annual_income'] = clean_currency_column(demographics['annual_income'])

# Standardize employment type
employment_mapping = {
    'full_time': 'full_time',
    'fulltime': 'full_time',
    'ft': 'full_time',
    'part_time': 'part_time',
    'pt': 'part_time',
    'self_employed': 'self_employed',
    'self_emp': 'self_employed',
    'contract': 'contract',
    'contractor': 'contract'
}
demographics['employment_type'] = standardize_categorical(
    demographics['employment_type'], 
    employment_mapping
)

# Standardize education levels
education_mapping = {
    'high_school': 'high_school',
    'some_college': 'some_college',
    'bachelor': 'bachelor',
    'graduate': 'graduate',
    'advanced': 'advanced'
}
demographics['education'] = standardize_categorical(
    demographics['education'],
    education_mapping
)

# Handle employment length missing values
demographics['employment_length_missing'] = (
    demographics['employment_length'].isna().astype(int)
)
# Impute with median better for data with outliers
median_emp_len = demographics['employment_length'].median()
demographics['employment_length'] = (
    demographics['employment_length']
    .fillna(median_emp_len)
    .astype('Float64')
)

# Cheking age range
n_invalid_age = (demographics['age'] > 100).sum()
if n_invalid_age > 0:
    print(f"{n_invalid_age} records with age > 100")

report_data_quality(demographics, "Demographics")

demographics.head()


Data quality report: Demographics
Shape: 89,999 rows × 9 columns

Missing Values:
No missing values



,customer_id,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents,employment_length_missing
0,10000,41,61800,2.2,full_time,graduate,Married,2,0
1,10001,38,28600,7.0,full_time,high_school,Married,0,0
2,10002,18,20700,0.8,full_time,bachelor,Single,0,0
3,10003,27,31400,4.8,full_time,bachelor,Single,0,0
4,10004,26,24600,5.2,full_time,high_school,Single,0,0


## 5. Credit History Cleaning

In [19]:
credit_history = pd.read_parquet('credit_history.parquet')
credit_history = credit_history.rename(columns={'customer_number': 'customer_id'})


# most customers have 0-2 delinquencies, distribution is skewed, so we are using median
median_delinq = credit_history['num_delinquencies_2yrs'].median()
credit_history['num_delinquencies_2yrs'] = (
    credit_history['num_delinquencies_2yrs'].fillna(median_delinq)
)

report_data_quality(credit_history, "Credit History")

credit_history.head()


Data quality report: Credit History
Shape: 89,999 rows × 12 columns

Missing Values:
No missing values



,customer_id,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,num_delinquencies_2yrs,num_inquiries_6mo,recent_inquiry_count,num_public_records,num_collections,account_diversity_index
0,10000,696,14,22.8,273.6,169100.0,0.0,2,2,1,0,0.499
1,10001,659,13,3.5,42.0,78200.0,0.0,6,6,0,0,0.298
2,10002,662,3,0.0,0.0,41400.0,0.0,2,2,0,0,0.174
3,10003,676,8,9.0,108.0,60000.0,0.0,1,1,0,0,0.263
4,10004,678,7,8.0,96.0,49700.0,0.0,1,1,0,0,0.298


## 6. Loan Details Cleaning

In [20]:
loan_details = pd.read_excel('loan_details.xlsx')

loan_type_mapping = {
    'personal': 'personal',
    'personal_loan': 'personal',
    'credit_card': 'credit_card',
    'creditcard': 'credit_card',
    'cc': 'credit_card',
    'mortgage': 'mortgage',
    'home_loan': 'mortgage'
}
loan_details['loan_type'] = standardize_categorical(
    loan_details['loan_type'],
    loan_type_mapping
)

loan_details['loan_amount'] = clean_currency_column(loan_details['loan_amount'])

report_data_quality(loan_details, "Loan Details")

print(f"\nLoan Type Distribution:")
print(loan_details['loan_type'].value_counts())

loan_details.head()


Data quality report: Loan Details
Shape: 89,999 rows × 10 columns

Missing Values:
No missing values


Loan Type Distribution:
loan_type
personal       36232
mortgage       31429
credit_card    22338
Name: count, dtype: int64


,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,10000,personal,17700,36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W
1,10001,mortgage,114000,180,6.83,Refinance,0.774,Branch,1011,B
2,10002,personal,9300,36,13.99,Major Purchase,0.000,Online,1084,K
3,10003,personal,8700,48,13.26,Medical,0.000,Online,1048,A
4,10004,personal,7200,24,10.77,Debt Consolidation,0.000,Branch,1055,S


## 7. Financial Ratios Cleaning

In [7]:
data = []
with open('financial_ratios.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

financial_ratios = pd.DataFrame(data)
financial_ratios = financial_ratios.rename(columns={'cust_num': 'customer_id'})

numeric_cols = [col for col in financial_ratios.columns if col != 'customer_id']

for col in numeric_cols:
    financial_ratios[col] = clean_currency_column(
        financial_ratios[col], 
        decimal_places=3
    )

# revolving_balance and credit_usage_amount are identical where revolving_balance exists
mask_non_null = financial_ratios['revolving_balance'].notna()
if mask_non_null.sum() > 0:
    are_equal = (
        financial_ratios.loc[mask_non_null, 'revolving_balance']
        .sub(financial_ratios.loc[mask_non_null, 'credit_usage_amount'])
        .abs() < 1e-6
    ).mean()
    print(f"Revolving balance and credit usage are identical: {are_equal:.1%}")

# Create missingness indicator for revolving_balance
financial_ratios['revolving_balance_missing'] = (
    financial_ratios['revolving_balance'].isna().astype(int)
)

financial_ratios = financial_ratios.drop(columns=['revolving_balance'])

report_data_quality(financial_ratios, "Financial Ratios")

financial_ratios.head()

Revolving balance and credit usage are identical: 100.0%

Data quality report: Financial Ratios
Shape: 89,999 rows × 16 columns

Missing Values:
No missing values



,customer_id,monthly_income,existing_monthly_debt,monthly_payment,debt_to_income_ratio,debt_service_ratio,payment_to_income_ratio,credit_utilization,credit_usage_amount,available_credit,total_monthly_debt_payment,annual_debt_payment,loan_to_annual_income,total_debt_amount,monthly_free_cash_flow,revolving_balance_missing
0,10000,5150.00,738.64,592.13,0.258,0.258,0.115,0.841,142213.1,26886.9,1330.77,15969.24,0.286,159913.1,3819.230,0
1,10001,2383.33,392.21,1013.86,0.590,0.590,0.425,0.971,75932.2,2267.8,1406.07,16872.84,3.986,189932.2,977.260,0
2,10002,1725.00,204.07,317.81,0.303,0.303,0.184,0.539,22314.6,19085.4,521.88,6262.56,0.449,31614.6,1203.120,0
3,10003,2616.67,288.71,234.52,0.200,0.200,0.090,0.147,8820.0,51180.0,523.23,6278.76,0.277,17520.0,2093.437,0
4,10004,2050.00,248.77,334.81,0.285,0.285,0.163,0.488,24253.6,25446.4,583.58,7002.96,0.293,31453.6,1466.420,0


## 8. Geographic Data Cleaning

In [8]:
geographic_data = pd.read_xml('geographic_data.xml')

geographic_data = geographic_data.rename(columns={'id': 'customer_id'})

int_cols = ['regional_median_rent', 'housing_price_index', 'cost_of_living_index']
for col in int_cols:
    geographic_data[col] = pd.to_numeric(
        geographic_data[col], 
        errors='coerce'
    ).astype('Int64')

report_data_quality(geographic_data, "Geographic Data")

geographic_data.head()


Data quality report: Geographic Data
Shape: 89,999 rows × 8 columns

Missing Values:
No missing values



,customer_id,state,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code
0,10000,OH,4.8,56000,1380,91,73,451
1,10001,PA,4.4,61000,1510,92,87,537
2,10002,VA,3.9,74000,1920,125,103,679
3,10003,CA,5.8,75000,1690,158,121,719
4,10004,WA,5.8,78000,1700,152,127,933


## 9. Export Cleaned Datasets

In [9]:
output_files = {
    'metadata_cleaned.csv': metadata,
    'demographics_cleaned.csv': demographics,
    'credit_history_cleaned.csv': credit_history,
    'loan_details_cleaned.csv': loan_details,
    'financial_ratios_cleaned.csv': financial_ratios,
    'geographic_data_cleaned.csv': geographic_data
}

print("Exporting cleaned datasets...")
for filename, dataframe in output_files.items():
    dataframe.to_csv(filename, index=False)
    print(f"{filename} - {len(dataframe):,} rows")

print("\ncleaned and exported successfully!")

Exporting cleaned datasets...
metadata_cleaned.csv - 89,999 rows
demographics_cleaned.csv - 89,999 rows
credit_history_cleaned.csv - 89,999 rows
loan_details_cleaned.csv - 89,999 rows
financial_ratios_cleaned.csv - 89,999 rows
geographic_data_cleaned.csv - 89,999 rows

cleaned and exported successfully!


In [10]:
def merge_with_tracking(left_df, right_df, on, how='left', suffix='', dataset_name=''):

    initial_rows = len(left_df)
    initial_cols = len(left_df.columns)
    
    if suffix:
        merged = left_df.merge(right_df, on=on, how=how, suffixes=('', suffix))
    else:
        merged = left_df.merge(right_df, on=on, how=how)
    
    new_cols = len(merged.columns) - initial_cols
    matched_records = right_df[on].isin(left_df[on]).sum()
    match_rate = (matched_records / len(right_df) * 100) if len(right_df) > 0 else 0
    
    print(f"\n{'─'*60}")
    print(f"Merged: {dataset_name}")
    print(f"{'─'*60}")
    print(f"  Rows before: {initial_rows:,}")
    print(f"  Rows after:  {len(merged):,}")
    print(f"  New columns: {new_cols}")
    print(f"  Match rate:  {match_rate:.2f}% ({matched_records:,}/{len(right_df):,})")
    
    return merged

print("Function defined")

Function defined


## 10. Merging cleaned datasets

In [21]:
# application metadata is base
print(f"\nBase dataset: Application Metadata ({len(metadata):,} rows)")

master_df = metadata.copy()

# Merge demographics
master_df = merge_with_tracking(
    master_df, 
    demographics, 
    on='customer_id',
    dataset_name='Demographics'
)

# Merge credit history
master_df = merge_with_tracking(
    master_df,
    credit_history,
    on='customer_id',
    dataset_name='Credit History'
)

# Merge loan details
master_df = merge_with_tracking(
    master_df,
    loan_details,
    on='customer_id',
    dataset_name='Loan Details'
)

# Merge financial ratios
master_df = merge_with_tracking(
    master_df,
    financial_ratios,
    on='customer_id',
    dataset_name='Financial Ratios'
)

# Merge geographic data
master_df = merge_with_tracking(
    master_df,
    geographic_data,
    on='customer_id',
    dataset_name='Geographic Data'
)

print("\n" + "="*60)
print(f"Main dataset has: {len(master_df):,} rows × {len(master_df.columns)} columns")
print("="*60)


Base dataset: Application Metadata (89,999 rows)

────────────────────────────────────────────────────────────
Merged: Demographics
────────────────────────────────────────────────────────────
  Rows before: 89,999
  Rows after:  89,999
  New columns: 8
  Match rate:  100.00% (89,999/89,999)

────────────────────────────────────────────────────────────
Merged: Credit History
────────────────────────────────────────────────────────────
  Rows before: 89,999
  Rows after:  89,999
  New columns: 11
  Match rate:  100.00% (89,999/89,999)

────────────────────────────────────────────────────────────
Merged: Loan Details
────────────────────────────────────────────────────────────
  Rows before: 89,999
  Rows after:  89,999
  New columns: 9
  Match rate:  100.00% (89,999/89,999)

────────────────────────────────────────────────────────────
Merged: Financial Ratios
────────────────────────────────────────────────────────────
  Rows before: 89,999
  Rows after:  89,999
  New columns: 15
  Mat

## 11. Dataset Quality Report

In [22]:
print("Dataset quality report")

print(f"\nDataset Dimensions:")
print(f"  Rows: {len(master_df):,}")
print(f"  Columns: {len(master_df.columns)}")
print(f"  Memory: {master_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

total_cells = len(master_df) * len(master_df.columns)
missing_cells = master_df.isnull().sum().sum()
completeness = (1 - missing_cells / total_cells) * 100

print(f"\nData Completeness:")
print(f"  Complete: {total_cells - missing_cells:,} / {total_cells:,} cells")
print(f"  Missing: {missing_cells:,} cells")
print(f"  Completeness Rate: {completeness:.2f}%")

missing_by_col = master_df.isnull().sum()
cols_with_missing = missing_by_col[missing_by_col > 0].sort_values(ascending=False)

if len(cols_with_missing) > 0:
    print(f"\nColumns with Missing Values: {len(cols_with_missing)}")
    for col, count in cols_with_missing.head(10).items():
        pct = (count / len(master_df) * 100)
        print(f"  {col}: {count:,} ({pct:.2f}%)")
    if len(cols_with_missing) > 10:
        print(f"  ... and {len(cols_with_missing) - 10} more")
else:
    print(f"\nNo missing values found")

print(f"\nData Types:")
dtype_counts = master_df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  {dtype}: {count} columns")

n_duplicates = master_df.duplicated().sum()
print(f"\nDuplicate Rows: {n_duplicates:,}")

if 'default' in master_df.columns:
    print(f"\nTarget Variable Distribution:")
    target_counts = master_df['default'].value_counts().sort_index()
    for val, count in target_counts.items():
        pct = (count / len(master_df) * 100)
        print(f"  Class {val}: {count:,} ({pct:.2f}%)")
    
    if len(target_counts) == 2:
        imbalance = target_counts.max() / target_counts.min()
        print(f"  Imbalance Ratio: {imbalance:.2f}:1")

print("Quality check complete")

master_df.head(3)

Dataset quality report

Dataset Dimensions:
  Rows: 89,999
  Columns: 64
  Memory: 93.24 MB

Data Completeness:
  Complete: 5,759,936 / 5,759,936 cells
  Missing: 0 cells
  Completeness Rate: 100.00%

No missing values found

Data Types:
  float64: 23 columns
  Int64: 14 columns
  object: 12 columns
  int64: 12 columns
  int32: 2 columns
  Float64: 1 columns

Duplicate Rows: 0

Target Variable Distribution:
  Class 0: 85,405 (94.90%)
  Class 1: 4,594 (5.10%)
  Imbalance Ratio: 18.59:1
Quality check complete


,customer_id,application_id,application_hour,application_day_of_week,account_open_year,preferred_contact,referral_code,account_status_code,random_noise_1,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing,default,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents,employment_length_missing,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,num_delinquencies_2yrs,num_inquiries_6mo,recent_inquiry_count,num_public_records,num_collections,account_diversity_index,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign,monthly_income,existing_monthly_debt,monthly_payment,debt_to_income_ratio,debt_service_ratio,payment_to_income_ratio,credit_utilization,credit_usage_amount,available_credit,total_monthly_debt_payment,annual_debt_payment,loan_to_annual_income,total_debt_amount,monthly_free_cash_flow,revolving_balance_missing,state,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code
0,10000,620515,5,6,2013,Mail,REF0000,ACT-2,1.137,13,2,1,1,0,41,61800,2.2,full_time,graduate,Married,2,0,696,14,22.8,273.6,169100.0,0.0,2,2,1,0,0.499,personal,17700,36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W,5150.00,738.64,592.13,0.258,0.258,0.115,0.841,142213.1,26886.9,1330.77,15969.24,0.286,159913.1,3819.23,0,OH,4.8,56000,1380,91,73,451
1,10001,624978,4,2,2015,Phone,REF0000,ACT-3,-0.165,6,1,1,1,1,38,28600,7.0,full_time,high_school,Married,0,0,659,13,3.5,42.0,78200.0,0.0,6,6,0,0,0.298,mortgage,114000,180,6.83,Refinance,0.774,Branch,1011,B,2383.33,392.21,1013.86,0.590,0.590,0.425,0.971,75932.2,2267.8,1406.07,16872.84,3.986,189932.2,977.26,0,PA,4.4,61000,1510,92,87,537
2,10002,564658,10,3,2020,Phone,REF0000,ACT-3,0.527,1,2,1,0,0,18,20700,0.8,full_time,bachelor,Single,0,0,662,3,0.0,0.0,41400.0,0.0,2,2,0,0,0.174,personal,9300,36,13.99,Major Purchase,0.000,Online,1084,K,1725.00,204.07,317.81,0.303,0.303,0.184,0.539,22314.6,19085.4,521.88,6262.56,0.449,31614.6,1203.12,0,VA,3.9,74000,1920,125,103,679


## 12. Export Main Dataset

In [23]:
# Export main dataset
master_df.to_csv('main_dataset.csv', index=False)
print(f"Master dataset exported: main_dataset.csv")
print(f"Rows: {len(master_df):,}, Columns: {len(master_df.columns)}")

Master dataset exported: main_dataset.csv
Rows: 89,999, Columns: 64
